## Imports

In [402]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt 
from sklearn.decomposition import PCA
from tqdm import tqdm
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix 

from IPython.display import display

sns.set_style('whitegrid')
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load Data

In [403]:
inter_obs_params = ['var_inter_prob' , 'var_inter_index' , 'var_inter_sigma']

In [381]:
df = pd.read_csv('../processed_data/v3/47_tuc_BH_rf_impute_no_norm.csv' ,index_col='obs_id').sort_index()
display(df)
df_bh =  df.drop(columns=inter_obs_params)
df = pd.read_csv('../processed_data/v3/47_tuc_CV_rf_impute_no_norm.csv' , index_col='obs_id').sort_index()
df_cv = df.drop(columns=inter_obs_params)
df = pd.read_csv('../processed_data/v3/47_tuc_NS_rf_impute_no_norm.csv' ,index_col='obs_id').sort_index()
df_ns = df.drop(columns=inter_obs_params)
df = pd.read_csv('../processed_data/v3/47_tuc_PULSAR_rf_impute_no_norm.csv' , index_col='obs_id').sort_index()
df_plsr = df.drop(columns=inter_obs_params)
df_arr = [df_cv , df_bh , df_ns , df_plsr] 


,class,src_n,src_id,significance,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,...,var_inter_prob,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
obs_id,,,,,,,,,,,,,,,,,,,,,
TUC_OBS_0,TUC,CXOGLB J002353.9-720350,TC0001,29.48,-6.101714,-5.544698,-5.088949,-6.269453,-5.928118,-5.500313,...,0.74580,-0.8688,0.1468,-0.4703,-1.0000,-0.2430,-0.7651,-0.2311,-0.9994,-0.5334
TUC_OBS_1,TUC,CXOGLB J002353.9-720350,TC0001,29.48,-5.848018,-5.198391,-4.459796,-6.053843,-5.708409,-5.297311,...,0.74580,-0.5141,0.1318,-0.4229,-0.8151,-0.2867,-0.6690,-0.1924,-0.6384,-0.4922
TUC_OBS_10,TUC,CXOGLB J002400.9-720453,TC0002,61.02,-4.899974,-3.615647,-3.847406,-4.940437,-4.352031,-3.898253,...,1.00000,-0.6115,-0.5415,-0.8139,-0.8451,-0.4641,-0.5041,-0.5759,-0.8295,-0.4847
TUC_OBS_100,TUC,CXOGLB J002404.6-720453,TC0012,8.61,-6.952336,-6.305219,-5.674279,-7.262731,-6.781990,-6.034563,...,0.34260,-1.0000,0.0693,-0.7027,-1.0000,-0.5665,-0.9550,-0.9994,-0.9994,-0.8007
TUC_OBS_1000,TUC,CXOGLB J002406.8-720512,TC0103,11.07,-6.801893,-5.877129,-6.729954,-7.070734,-6.297311,-6.071092,...,0.69500,-0.8376,-0.3354,-0.5765,-0.9163,-0.0906,-0.4516,-0.6059,-0.7658,-0.2773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TUC_OBS_989,TUC,CXOGLB J002406.5-720430,TC0102,3.41,-6.856985,-6.319211,-6.733523,-7.116566,-6.536406,-6.526951,...,0.17125,-0.7314,-0.0806,-0.0643,-0.7464,0.2904,-0.2804,-0.4172,-0.4260,-0.0012
TUC_OBS_99,TUC,CXOGLB J002404.6-720453,TC0012,8.61,-6.022459,-6.340084,-5.044683,-6.234783,-5.961380,-5.676954,...,0.34260,-1.0000,0.2992,-0.1393,-1.0000,-0.0743,-1.0000,-0.9994,-0.9994,-0.9994
TUC_OBS_995,TUC,CXOGLB J002406.8-720512,TC0103,11.07,-6.288783,-5.928486,-5.874950,-6.486250,-6.274252,-6.085181,...,0.69500,-0.3891,0.3129,0.0506,-0.6040,0.0631,-0.5303,-0.0350,-0.2986,-0.2548


In [406]:
df = pd.read_csv('../processed_data/v3/all_once_rf_impute.csv' , index_col='obs_id').sort_index()
df = df.drop(columns=inter_obs_params)
df =  df[df['class']=='TUC']
df 

,class,src_n,src_id,significance,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,...,var_min,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
obs_id,,,,,,,,,,,,,,,,,,,,,
TUC_OBS_0,TUC,CXOGLB J002353.9-720350,TC0001,29.48,-0.548758,-0.102362,0.282133,-0.608158,-0.146392,0.172051,...,-0.485245,-1.457091,-0.555611,-1.125155,-1.172985,-0.685464,-0.749049,-0.679391,-1.364656,-0.562353
TUC_OBS_1,TUC,CXOGLB J002353.9-720350,TC0001,29.48,-0.255759,0.216102,1.015333,-0.300601,0.121838,0.423107,...,-0.459389,-0.774481,-0.588765,-1.042238,-0.871840,-0.778341,-0.534309,-0.614859,-0.832267,-0.487700
TUC_OBS_10,TUC,CXOGLB J002400.9-720453,TC0002,61.02,0.839154,1.671587,1.728998,0.904004,1.777764,2.153343,...,-0.444206,-0.961924,-2.076904,-1.726217,-0.920701,-1.155376,-0.165833,-1.254336,-1.114094,-0.474111
TUC_OBS_100,TUC,CXOGLB J002404.6-720453,TC0012,8.61,-1.531157,-0.801734,-0.399998,-1.607656,-1.188836,-0.488664,...,-0.276369,-1.709581,-0.726903,-1.531694,-1.172985,-1.373011,-1.173390,-1.960512,-1.364656,-1.046690
TUC_OBS_102,TUC,CXOGLB J002404.6-720453,TC0012,8.61,-1.165612,-0.337614,-0.399759,-1.244106,-0.807702,-0.147195,...,-0.485760,-1.017156,-0.381667,-1.494434,-1.111909,-1.378325,-1.003117,-0.652378,-1.139902,-0.951743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TUC_OBS_91,TUC,CXOGLB J002404.1-720420,TC0011,5.29,-1.633950,-1.085229,-1.079774,-1.651772,-1.501570,-1.088114,...,-0.511749,-0.935559,0.269905,-0.987484,-1.172985,-0.826161,-0.975185,-0.110782,-1.364656,-0.725429
TUC_OBS_92,TUC,CXOGLB J002404.1-720420,TC0011,5.29,-1.209550,-0.793118,-1.172270,-1.226226,-1.244984,-0.863108,...,-0.504873,-0.156533,0.499105,-0.535114,-0.719232,-0.889922,-0.844018,0.143342,-0.541149,-0.673245
TUC_OBS_93,TUC,CXOGLB J002404.1-720420,TC0011,5.29,-1.253559,-1.033029,-1.075851,-1.266002,-1.085101,-1.499236,...,-0.253495,-0.584535,-0.097212,0.762872,0.043647,1.072191,0.554361,-0.321050,0.407712,0.773061


In [382]:
info_col = ['src_n' , 'src_id' , 'significance' , 'class']
x_val_arr = []
id_data_arr  = []
for d in df_arr:
    x_val = d.copy()
    y_val = x_val['class']
    id_data = x_val[info_col]
    x_val = x_val.drop(columns=info_col)
    x_val_arr.append(x_val)
    id_data_arr.append(id_data)
x_val_arr[0] 

,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,photflux_aper_lolim_m,photflux_aper_b,photflux_aper_m,photflux_aper_lolim_u,...,var_min,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
obs_id,,,,,,,,,,,,,,,,,,,,,
TUC_OBS_0,-6.101714,-5.544698,-5.088949,-6.268046,-5.928118,-5.500313,-6.538501,-5.355660,-6.133831,-6.219251,...,0.000461,-0.8688,0.1468,-0.4703,-1.0000,-0.2430,-0.7651,-0.2311,-0.9994,-0.5334
TUC_OBS_1,-5.848018,-5.198391,-4.459796,-6.053843,-5.708409,-5.297311,-6.099087,-5.082284,-5.856049,-4.702677,...,0.000787,-0.5141,0.1318,-0.4229,-0.8151,-0.2867,-0.6690,-0.1924,-0.6384,-0.4922
TUC_OBS_10,-4.899974,-3.615647,-3.847406,-4.940437,-4.352031,-3.898253,-4.389979,-3.604674,-4.370590,-3.901010,...,0.000979,-0.6115,-0.5415,-0.8139,-0.8451,-0.4641,-0.5041,-0.5759,-0.8295,-0.4847
TUC_OBS_100,-6.952336,-6.305219,-5.674279,-7.263061,-6.781990,-6.034563,-8.435097,-6.048080,-7.073400,-6.904831,...,0.000379,-1.0000,0.0693,-0.7027,-1.0000,-0.5665,-0.9550,-0.9994,-0.9994,-0.8007
TUC_OBS_1000,-6.801893,-5.877129,-6.739496,-7.070734,-6.297311,-6.071092,-6.575445,-5.779108,-6.410721,-7.837550,...,0.000208,-0.8376,-0.3354,-0.5765,-0.9163,-0.0906,-0.4516,-0.6059,-0.7658,-0.2773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TUC_OBS_989,-6.856985,-6.319211,-6.742204,-7.116566,-6.536406,-6.526951,-6.891773,-6.168578,-6.678816,-7.972178,...,0.000141,-0.7314,-0.0806,-0.0643,-0.7464,0.2904,-0.2804,-0.4172,-0.4260,-0.0012
TUC_OBS_99,-6.022459,-6.340084,-5.060882,-6.222815,-5.961380,-5.676954,-7.421410,-5.641114,-6.200927,-5.817067,...,0.001262,-1.0000,0.2992,-0.1393,-1.0000,-0.0743,-1.0000,-0.9994,-0.9994,-0.9994
TUC_OBS_995,-6.288783,-5.928486,-5.875199,-6.486250,-6.274252,-6.085181,-6.743763,-5.767258,-6.442613,-6.092976,...,0.000235,-0.3891,0.3129,0.0506,-0.6040,0.0631,-0.5303,-0.0350,-0.2986,-0.2548


In [383]:
original = pd.read_csv('../processed_data/TUC_.csv')

In [384]:
original[x_val_arr[0].columns.to_list()]

,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,photflux_aper_lolim_m,photflux_aper_b,photflux_aper_m,photflux_aper_lolim_u,...,var_min,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
0,-6.101714,-5.544698,-5.088949,NaN,-5.928118,-5.500313,-6.538501,-5.355660,-6.133831,-6.219251,...,0.000461,-0.8688,0.1468,-0.4703,-1.0000,-0.2430,-0.7651,-0.2311,-0.9994,-0.5334
1,-5.848018,-5.198391,-4.459796,-6.053843,-5.708409,-5.297311,-6.099087,-5.082284,-5.856049,-4.702677,...,0.000787,-0.5141,0.1318,-0.4229,-0.8151,-0.2867,-0.6690,-0.1924,-0.6384,-0.4922
2,-5.862646,-4.910095,-4.916856,-5.946537,-5.778586,-5.141824,-5.913640,-4.879755,-5.841035,-4.975514,...,0.001348,-0.2417,0.0144,-0.6527,-0.7639,-0.5965,-0.6952,-0.1124,-0.7096,-0.6458
3,-5.934420,-4.980884,-4.931443,-6.031470,-5.775208,-5.220476,-5.917574,-4.947306,-5.840734,-4.993962,...,0.001671,-0.3492,-0.0718,-0.6452,-0.7726,-0.5253,-0.6452,-0.2086,-0.7108,-0.5871
4,-5.939302,-4.989700,-4.993106,-6.033717,-5.840132,-5.220692,-5.986741,-4.957031,-5.907279,-5.057298,...,0.001588,-0.2804,-0.0031,-0.6527,-0.7739,-0.5790,-0.6889,-0.1412,-0.7146,-0.6346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2380,-5.801893,NaN,NaN,NaN,-5.991400,-5.674690,NaN,-5.721704,NaN,NaN,...,NaN,-0.1768,1.0000,0.2392,-1.0000,0.0468,-1.0000,0.9994,-0.9994,-0.9994
2381,-6.242225,NaN,NaN,NaN,-6.463063,-6.389233,NaN,NaN,NaN,NaN,...,NaN,-0.1318,1.0000,1.0000,-0.2367,1.0000,-1.0000,0.9994,0.9994,-0.9994
2382,-6.277613,NaN,NaN,NaN,-6.464706,-6.383735,NaN,NaN,NaN,NaN,...,NaN,-0.1768,1.0000,1.0000,-0.2592,1.0000,-1.0000,0.9994,0.9994,-0.9994
2383,-7.379136,-7.096313,NaN,NaN,-7.024063,-6.736838,-8.045275,-6.705093,-7.285586,NaN,...,0.000078,-1.0000,-0.1405,-0.4728,-1.0000,-0.0019,-0.7427,-0.9994,-0.9994,-0.4035


In [385]:
for i in range(4):
    display(x_val_arr[i])

,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,photflux_aper_lolim_m,photflux_aper_b,photflux_aper_m,photflux_aper_lolim_u,...,var_min,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
obs_id,,,,,,,,,,,,,,,,,,,,,
TUC_OBS_0,-6.101714,-5.544698,-5.088949,-6.268046,-5.928118,-5.500313,-6.538501,-5.355660,-6.133831,-6.219251,...,0.000461,-0.8688,0.1468,-0.4703,-1.0000,-0.2430,-0.7651,-0.2311,-0.9994,-0.5334
TUC_OBS_1,-5.848018,-5.198391,-4.459796,-6.053843,-5.708409,-5.297311,-6.099087,-5.082284,-5.856049,-4.702677,...,0.000787,-0.5141,0.1318,-0.4229,-0.8151,-0.2867,-0.6690,-0.1924,-0.6384,-0.4922
TUC_OBS_10,-4.899974,-3.615647,-3.847406,-4.940437,-4.352031,-3.898253,-4.389979,-3.604674,-4.370590,-3.901010,...,0.000979,-0.6115,-0.5415,-0.8139,-0.8451,-0.4641,-0.5041,-0.5759,-0.8295,-0.4847
TUC_OBS_100,-6.952336,-6.305219,-5.674279,-7.263061,-6.781990,-6.034563,-8.435097,-6.048080,-7.073400,-6.904831,...,0.000379,-1.0000,0.0693,-0.7027,-1.0000,-0.5665,-0.9550,-0.9994,-0.9994,-0.8007
TUC_OBS_1000,-6.801893,-5.877129,-6.739496,-7.070734,-6.297311,-6.071092,-6.575445,-5.779108,-6.410721,-7.837550,...,0.000208,-0.8376,-0.3354,-0.5765,-0.9163,-0.0906,-0.4516,-0.6059,-0.7658,-0.2773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TUC_OBS_989,-6.856985,-6.319211,-6.742204,-7.116566,-6.536406,-6.526951,-6.891773,-6.168578,-6.678816,-7.972178,...,0.000141,-0.7314,-0.0806,-0.0643,-0.7464,0.2904,-0.2804,-0.4172,-0.4260,-0.0012
TUC_OBS_99,-6.022459,-6.340084,-5.060882,-6.222815,-5.961380,-5.676954,-7.421410,-5.641114,-6.200927,-5.817067,...,0.001262,-1.0000,0.2992,-0.1393,-1.0000,-0.0743,-1.0000,-0.9994,-0.9994,-0.9994
TUC_OBS_995,-6.288783,-5.928486,-5.875199,-6.486250,-6.274252,-6.085181,-6.743763,-5.767258,-6.442613,-6.092976,...,0.000235,-0.3891,0.3129,0.0506,-0.6040,0.0631,-0.5303,-0.0350,-0.2986,-0.2548


,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,photflux_aper_lolim_m,photflux_aper_b,photflux_aper_m,photflux_aper_lolim_u,...,var_min,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
obs_id,,,,,,,,,,,,,,,,,,,,,
TUC_OBS_0,-6.101714,-5.544698,-5.088949,-6.269453,-5.928118,-5.500313,-6.538501,-5.355660,-6.133831,-6.219251,...,0.000461,-0.8688,0.1468,-0.4703,-1.0000,-0.2430,-0.7651,-0.2311,-0.9994,-0.5334
TUC_OBS_1,-5.848018,-5.198391,-4.459796,-6.053843,-5.708409,-5.297311,-6.099087,-5.082284,-5.856049,-4.702677,...,0.000787,-0.5141,0.1318,-0.4229,-0.8151,-0.2867,-0.6690,-0.1924,-0.6384,-0.4922
TUC_OBS_10,-4.899974,-3.615647,-3.847406,-4.940437,-4.352031,-3.898253,-4.389979,-3.604674,-4.370590,-3.901010,...,0.000979,-0.6115,-0.5415,-0.8139,-0.8451,-0.4641,-0.5041,-0.5759,-0.8295,-0.4847
TUC_OBS_100,-6.952336,-6.305219,-5.674279,-7.262731,-6.781990,-6.034563,-8.435097,-6.048080,-7.073400,-6.904831,...,0.000845,-1.0000,0.0693,-0.7027,-1.0000,-0.5665,-0.9550,-0.9994,-0.9994,-0.8007
TUC_OBS_1000,-6.801893,-5.877129,-6.729954,-7.070734,-6.297311,-6.071092,-6.575445,-5.779108,-6.410721,-7.864681,...,0.000208,-0.8376,-0.3354,-0.5765,-0.9163,-0.0906,-0.4516,-0.6059,-0.7658,-0.2773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TUC_OBS_989,-6.856985,-6.319211,-6.733523,-7.116566,-6.536406,-6.526951,-6.891773,-6.168578,-6.678816,-7.967275,...,0.000141,-0.7314,-0.0806,-0.0643,-0.7464,0.2904,-0.2804,-0.4172,-0.4260,-0.0012
TUC_OBS_99,-6.022459,-6.340084,-5.044683,-6.234783,-5.961380,-5.676954,-7.425689,-5.641114,-6.188738,-5.842052,...,0.001118,-1.0000,0.2992,-0.1393,-1.0000,-0.0743,-1.0000,-0.9994,-0.9994,-0.9994
TUC_OBS_995,-6.288783,-5.928486,-5.874950,-6.486250,-6.274252,-6.085181,-6.743763,-5.767258,-6.442613,-6.100781,...,0.000235,-0.3891,0.3129,0.0506,-0.6040,0.0631,-0.5303,-0.0350,-0.2986,-0.2548


,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,photflux_aper_lolim_m,photflux_aper_b,photflux_aper_m,photflux_aper_lolim_u,...,var_min,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
obs_id,,,,,,,,,,,,,,,,,,,,,
TUC_OBS_0,-6.101714,-5.544698,-5.088949,-6.271663,-5.928118,-5.500313,-6.538501,-5.355660,-6.133831,-6.219251,...,0.000461,-0.8688,0.1468,-0.4703,-1.0000,-0.2430,-0.7651,-0.2311,-0.9994,-0.5334
TUC_OBS_1,-5.848018,-5.198391,-4.459796,-6.053843,-5.708409,-5.297311,-6.099087,-5.082284,-5.856049,-4.702677,...,0.000787,-0.5141,0.1318,-0.4229,-0.8151,-0.2867,-0.6690,-0.1924,-0.6384,-0.4922
TUC_OBS_10,-4.899974,-3.615647,-3.847406,-4.940437,-4.352031,-3.898253,-4.389979,-3.604674,-4.370590,-3.901010,...,0.000979,-0.6115,-0.5415,-0.8139,-0.8451,-0.4641,-0.5041,-0.5759,-0.8295,-0.4847
TUC_OBS_100,-6.952336,-6.305219,-5.674279,-7.267389,-6.781990,-6.034563,-8.435097,-6.048080,-7.073400,-6.904831,...,0.000818,-1.0000,0.0693,-0.7027,-1.0000,-0.5665,-0.9550,-0.9994,-0.9994,-0.8007
TUC_OBS_1000,-6.801893,-5.877129,-6.737811,-7.070734,-6.297311,-6.071092,-6.575445,-5.779108,-6.410721,-7.859590,...,0.000208,-0.8376,-0.3354,-0.5765,-0.9163,-0.0906,-0.4516,-0.6059,-0.7658,-0.2773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TUC_OBS_989,-6.856985,-6.319211,-6.739598,-7.116566,-6.536406,-6.526951,-6.891773,-6.168578,-6.678816,-7.979791,...,0.000141,-0.7314,-0.0806,-0.0643,-0.7464,0.2904,-0.2804,-0.4172,-0.4260,-0.0012
TUC_OBS_99,-6.022459,-6.340084,-5.056488,-6.220290,-5.961380,-5.676954,-7.372429,-5.641114,-6.189700,-5.680564,...,0.001208,-1.0000,0.2992,-0.1393,-1.0000,-0.0743,-1.0000,-0.9994,-0.9994,-0.9994
TUC_OBS_995,-6.288783,-5.928486,-5.875316,-6.486250,-6.274252,-6.085181,-6.743763,-5.767258,-6.442613,-6.083417,...,0.000235,-0.3891,0.3129,0.0506,-0.6040,0.0631,-0.5303,-0.0350,-0.2986,-0.2548


,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,photflux_aper_lolim_m,photflux_aper_b,photflux_aper_m,photflux_aper_lolim_u,...,var_min,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
obs_id,,,,,,,,,,,,,,,,,,,,,
TUC_OBS_0,-6.101714,-5.544698,-5.088949,-6.261944,-5.928118,-5.500313,-6.538501,-5.355660,-6.133831,-6.219251,...,0.000461,-0.8688,0.1468,-0.4703,-1.0000,-0.2430,-0.7651,-0.2311,-0.9994,-0.5334
TUC_OBS_1,-5.848018,-5.198391,-4.459796,-6.053843,-5.708409,-5.297311,-6.099087,-5.082284,-5.856049,-4.702677,...,0.000787,-0.5141,0.1318,-0.4229,-0.8151,-0.2867,-0.6690,-0.1924,-0.6384,-0.4922
TUC_OBS_10,-4.899974,-3.615647,-3.847406,-4.940437,-4.352031,-3.898253,-4.389979,-3.604674,-4.370590,-3.901010,...,0.000979,-0.6115,-0.5415,-0.8139,-0.8451,-0.4641,-0.5041,-0.5759,-0.8295,-0.4847
TUC_OBS_100,-6.952336,-6.305219,-5.674279,-7.264692,-6.781990,-6.034563,-8.435097,-6.048080,-7.073400,-6.904831,...,0.001109,-1.0000,0.0693,-0.7027,-1.0000,-0.5665,-0.9550,-0.9994,-0.9994,-0.8007
TUC_OBS_1000,-6.801893,-5.877129,-6.736182,-7.070734,-6.297311,-6.071092,-6.575445,-5.779108,-6.410721,-7.879529,...,0.000208,-0.8376,-0.3354,-0.5765,-0.9163,-0.0906,-0.4516,-0.6059,-0.7658,-0.2773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TUC_OBS_989,-6.856985,-6.319211,-6.744856,-7.116566,-6.536406,-6.526951,-6.891773,-6.168578,-6.678816,-7.971056,...,0.000141,-0.7314,-0.0806,-0.0643,-0.7464,0.2904,-0.2804,-0.4172,-0.4260,-0.0012
TUC_OBS_99,-6.022459,-6.340084,-5.059632,-6.223717,-5.961380,-5.676954,-7.443143,-5.641114,-6.188326,-5.782396,...,0.001287,-1.0000,0.2992,-0.1393,-1.0000,-0.0743,-1.0000,-0.9994,-0.9994,-0.9994
TUC_OBS_995,-6.288783,-5.928486,-5.873513,-6.486250,-6.274252,-6.085181,-6.743763,-5.767258,-6.442613,-6.081509,...,0.000235,-0.3891,0.3129,0.0506,-0.6040,0.0631,-0.5303,-0.0350,-0.2986,-0.2548


In [386]:
x_val = sum([xi for xi in x_val_arr])/4
x_val

,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,photflux_aper_lolim_m,photflux_aper_b,photflux_aper_m,photflux_aper_lolim_u,...,var_min,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
obs_id,,,,,,,,,,,,,,,,,,,,,
TUC_OBS_0,-6.101714,-5.544698,-5.088949,-6.267776,-5.928118,-5.500313,-6.538501,-5.355660,-6.133831,-6.219251,...,0.000461,-0.8688,0.1468,-0.4703,-1.0000,-0.2430,-0.7651,-0.2311,-0.9994,-0.5334
TUC_OBS_1,-5.848018,-5.198391,-4.459796,-6.053843,-5.708409,-5.297311,-6.099087,-5.082284,-5.856049,-4.702677,...,0.000787,-0.5141,0.1318,-0.4229,-0.8151,-0.2867,-0.6690,-0.1924,-0.6384,-0.4922
TUC_OBS_10,-4.899974,-3.615647,-3.847406,-4.940437,-4.352031,-3.898253,-4.389979,-3.604674,-4.370590,-3.901010,...,0.000979,-0.6115,-0.5415,-0.8139,-0.8451,-0.4641,-0.5041,-0.5759,-0.8295,-0.4847
TUC_OBS_100,-6.952336,-6.305219,-5.674279,-7.264469,-6.781990,-6.034563,-8.435097,-6.048080,-7.073400,-6.904831,...,0.000788,-1.0000,0.0693,-0.7027,-1.0000,-0.5665,-0.9550,-0.9994,-0.9994,-0.8007
TUC_OBS_1000,-6.801893,-5.877129,-6.735861,-7.070734,-6.297311,-6.071092,-6.575445,-5.779108,-6.410721,-7.860337,...,0.000208,-0.8376,-0.3354,-0.5765,-0.9163,-0.0906,-0.4516,-0.6059,-0.7658,-0.2773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TUC_OBS_989,-6.856985,-6.319211,-6.740045,-7.116566,-6.536406,-6.526951,-6.891773,-6.168578,-6.678816,-7.972575,...,0.000141,-0.7314,-0.0806,-0.0643,-0.7464,0.2904,-0.2804,-0.4172,-0.4260,-0.0012
TUC_OBS_99,-6.022459,-6.340084,-5.055421,-6.225401,-5.961380,-5.676954,-7.415668,-5.641114,-6.191923,-5.780520,...,0.001219,-1.0000,0.2992,-0.1393,-1.0000,-0.0743,-1.0000,-0.9994,-0.9994,-0.9994
TUC_OBS_995,-6.288783,-5.928486,-5.874745,-6.486250,-6.274252,-6.085181,-6.743763,-5.767258,-6.442613,-6.089671,...,0.000235,-0.3891,0.3129,0.0506,-0.6040,0.0631,-0.5303,-0.0350,-0.2986,-0.2548


In [387]:
df1 = pd.DataFrame({
         'A':[4,5,4],
         'B':[7,8,90],
})

df2 = pd.DataFrame({
         'A':[4,50,4],
         'B':[7,8,9],
})

df3 = pd.DataFrame({
         'A':[4,5,4],
         'B':[7,8,9],
})

print ((df1+df2+df3)/3)

      A     B
0   4.0   7.0
1  20.0   8.0
2   4.0  36.0


In [388]:
df_bh = df.drop(columns=inter_obs_params)
df_bh 

,class,src_n,src_id,significance,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,...,var_min,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
obs_id,,,,,,,,,,,,,,,,,,,,,
TUC_OBS_0,TUC,CXOGLB J002353.9-720350,TC0001,29.48,-6.101714,-5.544698,-5.088949,-6.261944,-5.928118,-5.500313,...,0.000461,-0.8688,0.1468,-0.4703,-1.0000,-0.2430,-0.7651,-0.2311,-0.9994,-0.5334
TUC_OBS_1,TUC,CXOGLB J002353.9-720350,TC0001,29.48,-5.848018,-5.198391,-4.459796,-6.053843,-5.708409,-5.297311,...,0.000787,-0.5141,0.1318,-0.4229,-0.8151,-0.2867,-0.6690,-0.1924,-0.6384,-0.4922
TUC_OBS_10,TUC,CXOGLB J002400.9-720453,TC0002,61.02,-4.899974,-3.615647,-3.847406,-4.940437,-4.352031,-3.898253,...,0.000979,-0.6115,-0.5415,-0.8139,-0.8451,-0.4641,-0.5041,-0.5759,-0.8295,-0.4847
TUC_OBS_100,TUC,CXOGLB J002404.6-720453,TC0012,8.61,-6.952336,-6.305219,-5.674279,-7.264692,-6.781990,-6.034563,...,0.001109,-1.0000,0.0693,-0.7027,-1.0000,-0.5665,-0.9550,-0.9994,-0.9994,-0.8007
TUC_OBS_1000,TUC,CXOGLB J002406.8-720512,TC0103,11.07,-6.801893,-5.877129,-6.736182,-7.070734,-6.297311,-6.071092,...,0.000208,-0.8376,-0.3354,-0.5765,-0.9163,-0.0906,-0.4516,-0.6059,-0.7658,-0.2773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TUC_OBS_989,TUC,CXOGLB J002406.5-720430,TC0102,3.41,-6.856985,-6.319211,-6.744856,-7.116566,-6.536406,-6.526951,...,0.000141,-0.7314,-0.0806,-0.0643,-0.7464,0.2904,-0.2804,-0.4172,-0.4260,-0.0012
TUC_OBS_99,TUC,CXOGLB J002404.6-720453,TC0012,8.61,-6.022459,-6.340084,-5.059632,-6.223717,-5.961380,-5.676954,...,0.001287,-1.0000,0.2992,-0.1393,-1.0000,-0.0743,-1.0000,-0.9994,-0.9994,-0.9994
TUC_OBS_995,TUC,CXOGLB J002406.8-720512,TC0103,11.07,-6.288783,-5.928486,-5.873513,-6.486250,-6.274252,-6.085181,...,0.000235,-0.3891,0.3129,0.0506,-0.6040,0.0631,-0.5303,-0.0350,-0.2986,-0.2548


In [389]:
import joblib
clf  = joblib.load('forest/v2/all-clf-no-norm.joblib')

In [390]:
clf.predict(x_val)

array(['XRB', 'CV', 'PULSAR', ..., 'CV', 'XRB', 'CV'], dtype=object)

In [391]:
from sklearn.model_selection import validation_curve
from sklearn.ensemble import RandomForestClassifier 
clf = RandomForestClassifier()

In [392]:

clf = joblib.load('forest/v2/all-clf.joblib')
pred_cl = clf.predict(x_val)
pred_prob = [np.amax(pr) for pr in clf.predict_proba(x_val)]
id_data.insert(4 , 'pred_class' , pred_cl)
id_data.insert(5 , 'pred_prob' , pred_prob)
id_data =  id_data.sort_values(by='src_id')
id_data.to_csv('result/tuc_predictions.csv')

In [393]:
x_val[id_data['src_id']=='TC0001']

/home/kumaran/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,photflux_aper_lolim_m,photflux_aper_b,photflux_aper_m,photflux_aper_lolim_u,...,var_min,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
obs_id,,,,,,,,,,,,,,,,,,,,,
TUC_OBS_0,-6.101714,-5.544698,-5.088949,-6.267776,-5.928118,-5.500313,-6.538501,-5.355660,-6.133831,-6.219251,...,0.000461,-0.8688,0.1468,-0.4703,-1.0000,-0.2430,-0.7651,-0.2311,-0.9994,-0.5334
TUC_OBS_1,-5.848018,-5.198391,-4.459796,-6.053843,-5.708409,-5.297311,-6.099087,-5.082284,-5.856049,-4.702677,...,0.000787,-0.5141,0.1318,-0.4229,-0.8151,-0.2867,-0.6690,-0.1924,-0.6384,-0.4922
TUC_OBS_2,-5.862646,-4.910095,-4.916856,-5.946537,-5.778586,-5.141824,-5.913640,-4.879755,-5.841035,-4.975514,...,0.001348,-0.2417,0.0144,-0.6527,-0.7639,-0.5965,-0.6952,-0.1124,-0.7096,-0.6458
TUC_OBS_3,-5.934420,-4.980884,-4.931443,-6.031470,-5.775208,-5.220476,-5.917574,-4.947306,-5.840734,-4.993962,...,0.001671,-0.3492,-0.0718,-0.6452,-0.7726,-0.5253,-0.6452,-0.2086,-0.7108,-0.5871
TUC_OBS_4,-5.939302,-4.989700,-4.993106,-6.033717,-5.840132,-5.220692,-5.986741,-4.957031,-5.907279,-5.057298,...,0.001588,-0.2804,-0.0031,-0.6527,-0.7739,-0.5790,-0.6889,-0.1412,-0.7146,-0.6346
TUC_OBS_5,-6.113679,-5.030491,-4.924088,-6.223589,-5.794796,-5.266482,-5.928855,-4.997402,-5.856673,-4.980468,...,0.001435,-0.5265,-0.2567,-0.7252,-0.8389,-0.5041,-0.6227,-0.3910,-0.7845,-0.5646
TUC_OBS_6,-6.050805,-5.095988,-5.635090,-6.216643,-5.688246,-5.128427,-6.228192,-4.964170,-5.872895,-6.075310,...,0.001737,-1.0000,-0.2105,-0.7314,-1.0000,-0.4316,-0.7914,-0.9994,-0.9994,-0.6309


In [395]:
print(clf.predict(x_val[id_data['src_id']=='TC0005']))
print(clf.predict_proba(x_val[id_data['src_id']=='TC0005']))

['XRB' 'XRB' 'XRB' 'XRB' 'XRB' 'XRB' 'XRB' 'XRB' 'XRB' 'XRB' 'XRB']
[[0.14115571 0.41905471 0.43978958]
 [0.15086558 0.42155471 0.42757971]
 [0.14158496 0.41333534 0.44507971]
 [0.14976064 0.41905471 0.43118465]
 [0.14633683 0.42463644 0.42902674]
 [0.13997527 0.41744502 0.44257971]
 [0.14976064 0.42155471 0.42868465]
 [0.13587576 0.41480738 0.44931686]
 [0.1419807  0.41480738 0.44321192]
 [0.14358021 0.41494502 0.44147477]
 [0.13587576 0.41730738 0.44681686]]


/home/kumaran/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/home/kumaran/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [396]:
src = pd.read_csv('../source_list/TUC_data_clean_source_list.csv')
src = src[src['SRC_ID'].isin(np.unique(id_data['src_id']))]
src.set_index('SRC_ID')

,A_NAME,A_NAME_ALT\n,A_RA,A_DEC,A_SOURCE_TYPE,A_SOURCE_TYPE_FLAG,B_NAME,B_RA,B_DEC,XOFFSET_
SRC_ID,,,,,,,,,,
TC0001,CXOGLB J002353.9-720350,[GHE2001] W125,5.974829,-72.063922,QLX,?,2CXO J002353.9-720350,5.974992,-72.063947,0.003
TC0002,CXOGLB J002400.9-720453,[GHE2001] W58,6.003946,-72.081444,QLX,,2CXO J002400.9-720453,6.004075,-72.081474,0.003
TC0004,CXOGLB J002404.9-720451,[GHE2001] W37,6.020679,-72.080925,QLX,?,2CXO J002404.9-720451,6.020771,-72.080953,0.002
TC0005,CXOGLB J002408.3-720431,[GHE2001] W17,6.034604,-72.075397,QLX,?,2CXO J002408.3-720431,6.034720,-72.075417,0.002
TC0006,CXOGLB J002350.3-720431,[GHE2001] W108,5.959879,-72.075400,MSP,,2CXO J002350.3-720431,5.959967,-72.075434,0.003
...,...,...,...,...,...,...,...,...,...,...
TC0269,CXOGLB J002432.6-720622,[GHE2001] W209,6.136054,-72.106342,,,2CXO J002432.6-720622,6.135947,-72.106219,0.008
TC0270,CXOGLB J002434.1-720333,[GHE2001] W193,6.142454,-72.059333,,,2CXO J002434.1-720333,6.142426,-72.059317,0.001
TC0271,CXOGLB J002434.9-720501,[GHE2001] W132,6.146125,-72.083867,,,2CXO J002435.0-720501,6.146027,-72.083838,0.003


In [397]:
np.unique(src['A_SOURCE_TYPE'])

array(['             ', 'AB           ', 'AGB          ', 'CV           ',
       'MSP          ', 'QLX          '], dtype=object)

In [398]:
cv_ids = src[src['A_SOURCE_TYPE']=='             ']['SRC_ID'].to_list()
print(len(cv_ids))
len(id_data[id_data['src_id'].isin(cv_ids)])

134


670

In [399]:
from collections import Counter
src_pred = []
for i in np.unique(id_data['src_id'])[:]:
    p = id_data[id_data['src_id']==i]['pred_class'].to_list()
    p = Counter(p).most_common(1)
    src_pred.append(p[0][0])
src_pred = np.asarray(src_pred)
src_pred.shape

(255,)

In [400]:
res_df = pd.DataFrame()
res_df.insert(0 , 'SRC_ID' ,np.unique(id_data['src_id']))
res_df.insert(1 , 'src_pred' , src_pred)
res_df.set_index('SRC_ID')
res_df['src_pred'].value_counts()

XRB       246
PULSAR      9
Name: src_pred, dtype: int64

In [401]:
final = pd.concat([src , res_df] , axis=1)
final.to_csv('result/final.csv')